**Introduction**

PUBG is a battle royale game in which 100 players are dropped from an airplane. Once you are on the ground, you have to find weapons, ammo, and various other healing/booster items to stay alive. The main goal of this game is to be the last man standing.
Each PUBG match can be a solo, duo, or squads match. In a solo match, there can be maximum of 100 people on the map and the last man standing wins the game. In duos, there can be 50 groups max where each group can have a maximum of 2 members. In squads, every group can have maximum of 4 members so a total of 25 groups can be a part of a single match. All these numbers are the maximum numbers so there can be less than 25 groups in squads or less than 50 groups in duos. The rules are the same for every format; the last man standing (or the last group in case of squads/duos) wins the game.

**Analysis**

Since this is my first kernel and I am new to datascience, I will only perform the analysis of the training dataset. My analysis will only be for the solo games and my main goal will be to figure out the common traits among winners and losers. My focus will be on the trends of the top 10% players in every match and the last 25% in every game.

**Data**

Here is a list of every column of the data provided on kaggle and the description of every column.
•	DBNOs: Number of enemy players knocked.
•	Assists: Number of enemy players this player damaged that were killed by teammates.
•	Boosts: Number of boost items used.
•	damageDealt: Total damage dealt. Note: Self inflicted damage is subtracted.
•	headshotKills: Number of enemy players killed with headshots.
•	Heals: Number of healing items used.
•	killPlace: Ranking in match of number of enemy players killed.
•	killPoints: Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)
•	killStreaks: Max number of enemy players killed in a short amount of time.
•	Kills: Number of enemy players killed.
•	longestKill: Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
•	matchId: Integer ID to identify match. There are no matches that are in both the training and testing set.
•	Revives: Number of times this player revived teammates.
•	rideDistance: Total distance traveled in vehicles measured in meters.
•	roadKills: Number of kills while in a vehicle.
•	swimDistance: Total distance traveled by swimming measured in meters.
•	teamKills: Number of times this player killed a teammate.
•	vehicleDestroys: Number of vehicles destroyed.
•	walkDistance: Total distance traveled on foot measured in meters.
•	weaponsAcquired: Number of weapons picked up.
•	winPoints: Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)
•	groupId: Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
•	numGroups: Number of groups we have data for in the match.
•	maxPlace: Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
•	winPlacePerc: The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

df = pd.read_csv('../input/train.csv')
df.info()

# Any results you write to the current directory are saved as output.

In [ ]:
# Isolate the solo games
solo_df = pd.DataFrame()
solo_df = df[df["numGroups"]>50].copy()
solo_df.info()
# Checking if there aren't any duplicates or na's
solo_df.duplicated().value_counts()
solo_df.isna().any()

Double checking solo games. I selected numGroups greater than 50 because duos cannot have more than 50 groups. So, the records with numGroup greater than 50 must be a solo game. But I also want to double check. For this, I used revives because you cannot revive in the solos. I also know that there will be cases in duos where people couldn't revive but solos CANNOT have revives. So, if I have any revive in my selected dataframe then that means i made a mistake. So, let's check.

In [ ]:
solo_df["revives"].value_counts()
solo_df["DBNOs"].value_counts()

Since I am not going to use Id, groupId, DBNOs, teamkills, winPoints, numGroups, revives, and killPoints, I am going to delete these columns.

In [ ]:
solo_df.drop(["Id", "groupId", "DBNOs", "teamKills", "winPoints", "numGroups", "revives", "killPoints"], axis=1, inplace=True)

Now, let's have a look at the kill data.

In [ ]:
solo_df["kills"].describe()

Let's have a look at the guy who got the most kills in solo!

In [ ]:
solo_df[solo_df["kills"]==42]

Guy with the highest kills did not win the game :(

But let's see who actually won the game and how many kills did the winner have. I am going to use the matchId to get the game and look at the top 10 positions.

In [ ]:
solo_df[(solo_df["matchId"]==1441)&(solo_df["winPlacePerc"]>0.9)]

OMG, the one who won the game only had 2 kills :( But if you look at the highest kill guy, he had 42 roadkills... lol. So i guess he just ran around and killed people with his car...

Now,lets have a look at the guys with the highest killStreaks.

In [ ]:
solo_df["killStreaks"].describe()


In [ ]:
solo_df[solo_df["killStreaks"]==10]

Well, they did not win the game as well :( Well, just one of them did :(
Going to use their matchIds to look at their match winners. Pretty regular game except for the ride distances. 8 out of top 10 people rode tf out of the game tbh.

In [ ]:
solo_df[(solo_df["matchId"]==45605)&(solo_df["winPlacePerc"]>0.9)]

Most of these walked a lot though. Nothing major though.

In [ ]:
solo_df[(solo_df["matchId"]==21204)&(solo_df["winPlacePerc"]>0.9)]

So, the people with the highest kill and killstreaks don't win the game... interesting.
Now, let's have a look at the people who won the game without killing anyone.

In [ ]:
solo_df[(solo_df["kills"]==0)&(solo_df["winPlacePerc"]==1.0)]

Well, look at this. 8 lucky ducks won the game without killing anyone :D AND 3 of them DID NOT EVEN DO ANY DAMAGE :DDDDDD Majority of them rode and walked a lot and acquired quite a lot of weapons. So it's not like they could not do any damage. They just chose not to fight, i guess. Let's have a deeper look at how these people won the game. Were these campers? or runners?

In [ ]:
solo_df[solo_df["matchId"]==13392].describe()

In [ ]:
solo_df[(solo_df["matchId"]==13392)&(solo_df["winPlacePerc"]==1.0)]

After looking at the stats of all 8 people who won the game without any kills, none of the winners were at max in any field but all of them were above the average in the fields i was looking for (heals, boosts, walk distance, ride distance, weapons). 3 of them had less heals then the average whereas only one of them had less than average ride distance. So, it seems like majority of them weren't campers or runners. They were, i guess, just lucky? Also, since none of them were at the max of their fields, it seems like you don't have to be the best at everything. As long as you are a little above average, you have a pretty good chance of winning

Now, let's have a look at the kills of people who won the game against everyone else.

In [ ]:
solo_df["kills"][solo_df["winPlacePerc"]==1.0].describe()

In [ ]:
solo_df["kills"][solo_df["winPlacePerc"]<1.0].describe()

Interesting, but if you look at the count, people who did not win the game have a huge count. But this is also counting the number of 0 kills. So, we exclude the 0 kill count and lets see how the data changes

In [ ]:
solo_df["kills"][(solo_df["winPlacePerc"]<1.0)&(solo_df["kills"]>0.0)].describe()

As you can see, the results imporved but there still a huge kill difference between the winners and the losers. I also want to see if there is a difference in the top 10 and the winners.

In [ ]:
solo_df["kills"][(solo_df["winPlacePerc"]>=0.90)&(solo_df["winPlacePerc"]<1.0)&(solo_df["kills"]>0.0)].describe()

So it seems like winners do kill a lot more than losers. This also means that sitting or hiding isn't the best strategy and you should be good at killing if you want to win.
Speaking of killing, i also want to know why people die early in the game?

In [ ]:
solo_df["kills"].value_counts().head(10)

If you look at the count, majority of the people don't even get a single kill. Let's see if this trend changes when looking at the last 25%

In [ ]:
solo_df["kills"][(solo_df["winPlacePerc"]<=0.25)].value_counts()

So, it seems like the trend is still there in the last 25% people. Let's see if this is because majority of the people are afk or they can't find a weapon?

In [ ]:
solo_df["weaponsAcquired"][(solo_df["winPlacePerc"]<=0.25)].value_counts()

So it looks like majority of these actually do get a weapon. The reason why they die is either because they are unlucky or noob or the storm kills them. One way to figure this out is to look at the longestkill. People dying is close combats will obviously have a lower longestkill average

In [ ]:
solo_df["longestKill"][(solo_df["winPlacePerc"]<=0.25)].mean()

In [ ]:
solo_df["longestKill"][(solo_df["winPlacePerc"]>0.25)&(solo_df["winPlacePerc"]<=0.50)].mean()

So, i think majority of them die in close combats which also makes sense because majority of them acquire at least a single weapon. We can also confirm this with walk and ride distances

In [ ]:
plt.plot(solo_df["walkDistance"][(solo_df["winPlacePerc"]<=0.25)&(solo_df["walkDistance"]>0.0)].value_counts().sort_index().index.values,
        solo_df["walkDistance"][(solo_df["winPlacePerc"]<=0.25)&(solo_df["walkDistance"]>0.0)].value_counts().sort_index(), alpha=0.5, label="First 25%")
plt.plot(solo_df["walkDistance"][(solo_df["winPlacePerc"]>0.25)&(solo_df["winPlacePerc"]<=0.50)&(solo_df["walkDistance"]>0.0)].value_counts().sort_index().index.values,
        solo_df["walkDistance"][(solo_df["winPlacePerc"]>0.25)&(solo_df["winPlacePerc"]<=0.50)&(solo_df["walkDistance"]>0.0)].value_counts().sort_index(), alpha=0.5, label="25%-50%")
plt.legend()
plt.show()

In [ ]:
plt.plot(solo_df["rideDistance"][(solo_df["winPlacePerc"]<=0.25)&(solo_df["rideDistance"]>0.0)].value_counts().sort_index().index.values,
        solo_df["rideDistance"][(solo_df["winPlacePerc"]<=0.25)&(solo_df["rideDistance"]>0.0)].value_counts().sort_index(), alpha=0.5, label="First 25%")
plt.plot(solo_df["rideDistance"][(solo_df["winPlacePerc"]>0.25)&(solo_df["winPlacePerc"]<=0.50)&(solo_df["rideDistance"]>0.0)].value_counts().sort_index().index.values,
        solo_df["rideDistance"][(solo_df["winPlacePerc"]>0.25)&(solo_df["winPlacePerc"]<=0.50)&(solo_df["rideDistance"]>0.0)].value_counts().sort_index(), alpha=0.5, label="25%-50%")
plt.legend()
plt.show()

In [ ]:
solo_df["walkDistance"][(solo_df["winPlacePerc"]<=0.25)&(solo_df["rideDistance"]>0.0)].mean()

Having a mean walk distance of 155 m means that majority of the people don't even get out of their drop building. To give you some perspective, the small squares on your PUBG map  are 100m big. So, on average, 25% of the people don't even make it beyond the second square.
So, the bottom line is that majority of the people die within their drop zones. This makes sense because most people like to drop in hot zones where all the fun is and most of them die fighting. This also indicates that the weapon distribution is quite good since majority of the people end up with at least one weapon.

... to be continued